# Import Libraries

In [1]:
import pickle as pkl
import pandas as pd
import time
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

ModuleNotFoundError: No module named 'qwikidata'

# Set relevant parameters

In [2]:
properties = {"sexuality":"P91",
             "country":"P27",
             "language":"P6886",
             "religion":"P140",
             "gender":"P21",
             "ethnicity":"P172",
             "birthyear":"P569",
             "deathyear":"P570"}

# Read file

In [3]:
final_dataset = pd.read_csv("data/final_dataset_before_properties.csv", low_memory = False, index_col=0).fillna("")

In [4]:
final_dataset

,ISBN,title,author,alt_title,alt_author,correct_author,alt_first_author,viaf_id,QID
index,,,,,,,,,
2,0002005018,Clara Callan,Richard Bruce Wright,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406,Q7323887
3,0771597185,The teacher's daughter,Richard Bruce Wright,The Teacher's Daughter,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406,Q7323887
183415,0060506067,Clara Callan : A Novel,Richard B. Wright,Clara Callan,Richard B. Wright,Richard B. Wright,Richard B. Wright,64022406,Q7323887
183416,0060506075,Clara Callan : A Novel,Richard B. Wright,Clara Callan,Richard B. Wright,Richard B. Wright,Richard B. Wright,64022406,Q7323887
217653,000224408X,The age of longing: A novel,Richard B Wright,The Age of Longing,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406,Q7323887
...,...,...,...,...,...,...,...,...,...
269257,3453047192,Die amerikanische Zumutung: PlÃ¤doyers gegen d...,Rolf Winter,Die amerikanische Zumutung,Rolf Winter,Rolf Winter,Rolf Winter,39470984,Q104721
269260,0747204020,Illustrated Encyclopedia of Cacti,Glass Charles,The Illustrated Encyclopaedia of Cacti,Clive Innes|Charles Glass,Charles Glass,Clive Innes,86432382,Q5078055
269264,3525335423,Das Deutsche Kaiserreich 1871-1918.,Hans-Ulrich Wehler,"Das Deutsche Kaiserreich, 1871-1918",Hans-Ulrich Wehler,Hans-Ulrich Wehler,Hans-Ulrich Wehler,108252200,Q66155


In [5]:
unique_authors = final_dataset.drop_duplicates(["viaf_id","QID"])

In [6]:
len(unique_authors)

39642

In [5]:
unique_authors.to_csv("data/final_authors_before_properties.csv")

# Run Wikidata

In [6]:
for k,v in properties.items():
    unique_authors[k]=""

/export/scratch2/home/savvina/new_environment/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
start = time.time()
i=0
for index,row in unique_authors.iterrows():
    author_qid = row["QID"]
    author_dict = get_entity_dict_from_api(author_qid)
    
    try:
        assert author_dict["claims"]["P214"][0]["mainsnak"]["datavalue"]["value"]==row["viaf_id"]
    except:
        print(author_qid, row["viaf_id"])
        
    for prop, qid in properties.items():
        if prop=="birthyear" or prop=="deathyear":
            try:
                value = [x["mainsnak"]["datavalue"]["value"]["time"] for x in author_dict["claims"][qid]]
            except:
                value = ""
        else:
            try:
                value = [x["mainsnak"]["datavalue"]["value"]["id"] for x in author_dict["claims"][qid]]
            except:
                value = ""
        unique_authors.at[index,prop] = value
    if i%100==0:
        print("100 is gone!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", index)
        print(time.time()-start)
    if i%500==0:
        unique_authors.to_csv('data/final_authors_after_properties.csv')
    i+=1
unique_authors.to_csv('data/final_authors_after_properties.csv')

Q6151251 164842063
100 is gone!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 268729
0.2088935375213623
Q6175779 7583029
100 is gone!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 269101
31.931270837783813


In [14]:
unique_authors.country[2]

"['Q16']"